First, let's start with the dataset. I've randomly chosen ChocolateSales.csv because it seemed not so overwhelming and appropriate for the first attempts. You can download it right here https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales. It's free to use, so don't struggle :)

In the main.py you may find the final code, with the most important analysis and charts with no extra commands. So, feel free to explore.

 Let's start with importing the 'pandas' library:

In [1]:
import pandas as pd

Using pd as a shortcut for it as long as we are going to use it plenty of times.
So, our first task is to read a raw .csv file that we have downloaded:

In [8]:
data = pd.read_csv('./data/Chocolate Sales.csv')

Let's explore how our dataset looks like:

In [9]:
print(data.head())

     Sales Person    Country              Product       Date    Amount  \
0  Jehu Rudeforth         UK      Mint Chip Choco  04-Jan-22   $5,320    
1     Van Tuxwell      India        85% Dark Bars  01-Aug-22   $7,896    
2    Gigi Bohling      India  Peanut Butter Cubes  07-Jul-22   $4,501    
3    Jan Morforth  Australia  Peanut Butter Cubes  27-Apr-22  $12,726    
4  Jehu Rudeforth         UK  Peanut Butter Cubes  24-Feb-22  $13,685    

   Boxes Shipped  
0            180  
1             94  
2             91  
3            342  
4            184  


Doing great! Let's delete the NaN rows (if they exist):

In [10]:
data.dropna(inplace=True) #inplace=True means that we edit this exact object without creating new instances

Deleting spaces from columns' names for a more comfortable usage:

In [11]:
print(data.columns)
data.columns = data.columns.str.replace(' ', '', regex=False) #regex=False is used to treat the ' ' literally
print(data.columns)

Index(['Sales Person', 'Country', 'Product', 'Date', 'Amount',
       'Boxes Shipped'],
      dtype='object')
Index(['SalesPerson', 'Country', 'Product', 'Date', 'Amount', 'BoxesShipped'], dtype='object')


Let's parse the 'Date' column to the appropriate datetime64 format:

In [12]:
print(data['Date'].head(5))
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%y')
print(data['Date'].head(5))

0    04-Jan-22
1    01-Aug-22
2    07-Jul-22
3    27-Apr-22
4    24-Feb-22
Name: Date, dtype: object
0   2022-01-04
1   2022-08-01
2   2022-07-07
3   2022-04-27
4   2022-02-24
Name: Date, dtype: datetime64[ns]


Format used here is directly connected with the dates' format in the file. So, we adjust the parameter to correctly parse it to the needed one (datetime64).


Perfect! Preparations are finished! Let's get down to fetching some statistics!